In [1]:
# Adapted from LLama Factory
# https://colab.research.google.com/drive/1eRTPn37ltBbYsISy9Aw2NuI2Aq5CQrD9?usp=sharing
# Data formats
# https://github.com/hiyouga/LLaMA-Factory/tree/main/data

In [2]:
!which python
!which pip

~/.conda/envs/py310/bin/python
~/.conda/envs/py310/bin/pip


In [3]:
#!git clone --depth 1 https://github.com/PepeRulo/LLaMA-Factory.git

In [4]:
%cd /research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/LLaMA-Factory
%pwd

/research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/LLaMA-Factory


/home/jpastr08/.conda/envs/py310/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/LLaMA-Factory'

In [5]:
# Commented out to be able to `Run All Cells` smoothly.

#!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
#!pip uninstall -y jax
#!pip install -e .[torch,bitsandbytes,liger-kernel]
#!pip install huggingface_hub

#!pip freeze > actual_requirements.txt
#!cat actual_requirements.txt

In [6]:
import sys
import torch

print(sys.version)
print(torch.__version__)

3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
2.3.1+cu121


In [7]:
import psutil

# Get the total RAM in bytes
total_ram = psutil.virtual_memory().total

# Convert bytes to GB
total_ram_gb = total_ram / (1024 ** 3)

print(f"Total RAM: {total_ram_gb:.2f} GB")


Total RAM: 1507.35 GB


In [8]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

device = torch.device("cuda")

# Get the name of the GPU
gpu_name = torch.cuda.get_device_name(device)
print(f"GPU: {gpu_name}")

# You can also get more details
print(f"Number of GPUs: {torch.cuda.device_count()}")
print(f"Current CUDA device: {torch.cuda.current_device()}")
print(f"CUDA capability: {torch.cuda.get_device_capability(device)}")

print(torch.__version__)
print(torch.version.cuda)

!nvidia-smi

GPU: NVIDIA A100-SXM4-80GB
Number of GPUs: 8
Current CUDA device: 0
CUDA capability: (8, 0)
2.3.1+cu121
12.1
Tue Dec 10 04:56:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   41C    P0              63W / 400W |      5MiB / 81920MiB |      0%      Default |
|                                         |      

In [9]:
import os
from huggingface_hub import login
with open("../hf_token.txt", 'r') as file:
    token = file.readline().strip()
    login(token=token)

/home/jpastr08/.conda/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Don't forget to run this when there's been updates to the prompt dataset!
%cp /research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/files/dataset.json \
    ./data/dragen_alpaca.json

In [11]:
import json
with open("data/dataset_info.json", "r") as f:
    datasets = json.load(f)
    
# Per https://github.com/hiyouga/LLaMA-Factory/blob/main/data/README.md#alpaca-format
if "dragen_alpaca" not in datasets:
    datasets["dragen_alpaca"] = {
        "file_name": "dragen_alpaca.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    }
    
    with open("data/dataset_info.json", "w") as f:
        json.dump(datasets, f, indent=2, ensure_ascii=False)

In [12]:
## Run in terminal on the same node this OnDemand session is on.
# ssh <ondemand gpu node>
# module load conda3/202402
# conda activate <env>
## To run Gradio board (which by default launches on localhost:7860)
# cd /research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/LLaMA-Factory
# llamafactory-cli webui
## Run locally
# ssh -L <port shown above>:localhost:<port shown above> local_port_forwarding_test
## Open a browser locally to interface with Gradio board

In [13]:
# Run in terminal within same conda env as here.
# conda install nvidia/label/cuda-12.3.2::cuda-toolkit
# pip install deepspeed

In [14]:
# Training on Gradio board 
# See in:
# KIDS24-team12/vm_files/Jose/LLaMA-Factory/saves/Llama-3.1-70B-Instruct/lora/16_batch_size
# KIDS24-team12/vm_files/Jose/LLaMA-Factory/saves/Llama-3.1-70B-Instruct/lora/8_batch_size

In [21]:
# Infer base model with fine-tuned adapters

from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc

%cd /research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/LLaMA-Factory

args = dict(
  model_name_or_path="meta-llama/Meta-Llama-3.1-70B-Instruct",           # base model
  adapter_name_or_path="saves/Llama-3.1-70B-Instruct/lora/16_batch_size", # path to the LoRA adapters that were fine-tuned
  template="llama3",                                                     # same to the one in training
  finetuning_type="lora",                                                # same to the one in training
  quantization_bit=4,                                                    # load 4-bit quantized model
)
chat_model = ChatModel(args)

[INFO|configuration_utils.py:679] 2024-12-10 05:17:07,708 >> loading configuration file config.json from cache at /home/jpastr08/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-70B-Instruct/snapshots/945c8663693130f8be2ee66210e062158b2a9693/config.json
[INFO|configuration_utils.py:746] 2024-12-10 05:17:07,709 >> Model config LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3.1-70B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 28672,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 64,
  "num_hidden_layers": 80,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_fact

/research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/LLaMA-Factory


[INFO|tokenization_utils_base.py:2475] 2024-12-10 05:17:08,066 >> Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[INFO|configuration_utils.py:679] 2024-12-10 05:17:08,462 >> loading configuration file config.json from cache at /home/jpastr08/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-70B-Instruct/snapshots/945c8663693130f8be2ee66210e062158b2a9693/config.json
[INFO|configuration_utils.py:746] 2024-12-10 05:17:08,463 >> Model config LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3.1-70B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 28672,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num

[INFO|2024-12-10 05:17:08] llamafactory.data.template:157 >> Replace eos token: <|eot_id|>
[INFO|2024-12-10 05:17:08] llamafactory.data.template:157 >> Add pad token: <|eot_id|>


[INFO|configuration_utils.py:679] 2024-12-10 05:17:08,921 >> loading configuration file config.json from cache at /home/jpastr08/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-70B-Instruct/snapshots/945c8663693130f8be2ee66210e062158b2a9693/config.json
[INFO|configuration_utils.py:746] 2024-12-10 05:17:08,922 >> Model config LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3.1-70B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 28672,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 64,
  "num_hidden_layers": 80,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_fact

[INFO|2024-12-10 05:17:08] llamafactory.model.model_utils.quantization:157 >> Quantizing model to 4 bit with bitsandbytes.
[INFO|2024-12-10 05:17:08] llamafactory.model.patcher:157 >> Using KV cache for faster generation.


[INFO|modeling_utils.py:3937] 2024-12-10 05:17:08,927 >> loading weights file model.safetensors from cache at /home/jpastr08/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-70B-Instruct/snapshots/945c8663693130f8be2ee66210e062158b2a9693/model.safetensors.index.json
[INFO|modeling_utils.py:1670] 2024-12-10 05:17:08,936 >> Instantiating LlamaForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:1096] 2024-12-10 05:17:08,937 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ]
}


[INFO|modeling_utils.py:4800] 2024-12-10 05:17:59,908 >> All model checkpoint weights were used when initializing LlamaForCausalLM.

[INFO|modeling_utils.py:4808] 2024-12-10 05:17:59,909 >> All the weights of LlamaForCausalLM were initialized from the model checkpoint at meta-llama/Meta-Llama-3.1-70B-Instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already 

[INFO|2024-12-10 05:18:00] llamafactory.model.model_utils.attention:157 >> Using torch SDPA for faster training and inference.
[INFO|2024-12-10 05:18:03] llamafactory.model.adapter:157 >> Loaded adapter(s): saves/Llama-3.1-70B-Instruct/lora/16_batch_size
[INFO|2024-12-10 05:18:03] llamafactory.model.loader:157 >> all params: 70,605,479,936


In [ ]:
messages = []
print("Welcome to the CLI application, use `clear` to remove the history, use `exit` to exit the application.")
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("History has been removed.")
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)

  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

torch_gc()

Welcome to the CLI application, use `clear` to remove the history, use `exit` to exit the application.



User:  If the DRAGEN software is run with the parameter --enable-variant-caller=true, what are the implications? Construct a command line including this parameter


Assistant: When the DRAGEN software is run with the parameter --enable-variant-caller=true, it enables the variant caller, which is necessary for identifying genetic variants in the data. A command line with this parameter could look like: 

dragen -r /staging/human/reference/hg19/hg19.fa.k_21.f_16.m_149 \
--intermediate-results-dir /staging/examples/ \
--output-directory /staging/examples/ \
--output-file-prefix NA12878_e10_50M_30x_e10_50M \
--enable-map-align false \
--enable-map-align-output false \
--enable-sort true \
--enable-duplicate-marking true \
--enable-variant-caller true



User:  Using the Illumina DRAGEN software, how do you construct a command line for mapping FASTQ files to a reference genome?


Assistant: To map FASTQ files to a reference genome, the command line would include options like --ref-dir for the reference directory, --fastq-file1 and --fastq-file2 for the input FASTQ files, and --output-directory for the output location.


...

Assistant: To include parameters for setting the output directory to /data/output and specifying a memory limit of 16 GB, the command line would be: 

dragen -r /staging/human/reference/hg38/hg38.fa.k_21.f_16.m_149 \
--fastq-file1 /staging/examples/reads/SRA056922_30x_e10_50M_1.fastq \
--fastq-file2 /staging/examples/reads/SRA056922_30x_e10_50M_2.fastq \
--output-directory /data/output \
--output-file-prefix NA12878_e10_50M_30x_e10_50M \
--enable-map-align true \
--enable-map-align-output true \
--enable-sort true \
--enable-duplicate-marking true \
--enable-variant-caller true \
--enable-map-align-output true \
--enable-sort true \
--enable-duplicate-marking true \
--enable-variant-caller true \
--enable-map-align-output true \
--enable-sort true \
--enable-duplicate-marking true \
--enable-variant-caller true



User:  Write a command line to perform mapping, variant calling, and generate a BAM file, using DRAGEN. Include the input files reads_R1.fastq and reads_R2.fastq, a reference genome hg19, and an output directory /output_dir


Assistant: The command line to perform mapping, variant calling, and generate a BAM file, using DRAGEN, is:

dragen -r /staging/human/reference/hg19/hg19.fa.k_21.f_16.m_149 \
--fastq-file1 /staging/examples/reads/reads_R1.fastq \
--fastq-file2 /staging/examples/reads/reads_R2.fastq \
--output-directory /output_dir \
--output-file-prefix NA12878_e10_50M_30x_e10_50M \
--enable-map-align true \
--enable-map-align-output true \
--enable-sort true \
--enable-duplicate-marking true \
--enable-variant-caller true



User:  The following command results in an error: dragen --input sample.fastq --output-dir /output. Correct the command and explain the changes you made.


Assistant: The corrected command is: 



In [65]:
# Resources:
# https://www.reddit.com/r/LocalLLaMA/comments/1fps1cp/llama32_vs_llama31_in_medical_domain_llama31_70b/
# https://github.com/NVIDIA/RULER?tab=readme-ov-file